In [1]:
import pandas as pd 
import numpy as np

In [2]:
# Load the diabetes dataset
df = pd.read_csv('diabetes.csv')
print(f"Dataset loaded successfully! Shape: {df.shape}")

Dataset loaded successfully! Shape: (768, 9)


In [3]:
# Explore the first few rows of the dataset
print("First 5 rows of the dataset:")
df.head()

First 5 rows of the dataset:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Check correlation of features with the target variable (Outcome)
print("Correlation with target variable (Outcome):")
correlations = df.corr()['Outcome'].sort_values(key=abs, ascending=False)
print(correlations)

Correlation with target variable (Outcome):
Outcome                     1.000000
Glucose                     0.466581
BMI                         0.292695
Age                         0.238356
Pregnancies                 0.221898
DiabetesPedigreeFunction    0.173844
Insulin                     0.130548
SkinThickness               0.074752
BloodPressure               0.065068
Name: Outcome, dtype: float64


In [5]:
# Separate features (X) and target variable (y)
X = df.iloc[:,:-1].values  # All columns except the last one (features)
y = df.iloc[:,-1].values   # Last column (target: Outcome)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution: {np.bincount(y)}")  # Count of 0s and 1s

Features shape: (768, 8)
Target shape: (768,)
Target distribution: [500 268]


In [6]:
# Initialize StandardScaler for feature normalization
# Neural networks work better with normalized features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print("StandardScaler initialized successfully!")

StandardScaler initialized successfully!


In [7]:
# Scale the features to have mean=0 and std=1
# This helps neural networks train more effectively
X = scaler.fit_transform(X)
print("Features scaled successfully!")
print(f"Feature means after scaling: {X.mean(axis=0).round(6)}")
print(f"Feature std after scaling: {X.std(axis=0).round(6)}")

Features scaled successfully!
Feature means after scaling: [-0. -0.  0.  0. -0.  0.  0.  0.]
Feature std after scaling: [1. 1. 1. 1. 1. 1. 1. 1.]


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()

In [12]:
# Add layers to the model
# Using Input layer to avoid deprecation warning
from keras.layers import Input

model.add(Input(shape=(8,)))  # Input layer for 8 features
model.add(Dense(32, activation='relu'))  # Hidden layer with 32 neurons
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the baseline model
print("Training baseline model...")
history_baseline = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate baseline model
baseline_loss, baseline_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n=== Baseline Model Results ===")
print(f"Baseline Test Loss: {baseline_loss:.4f}")
print(f"Baseline Test Accuracy: {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)")

Training baseline model...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3299 - loss: 0.8366 - val_accuracy: 0.3821 - val_loss: 0.7733
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3299 - loss: 0.8366 - val_accuracy: 0.3821 - val_loss: 0.7733
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4440 - loss: 0.7494 - val_accuracy: 0.5122 - val_loss: 0.7114
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4440 - loss: 0.7494 - val_accuracy: 0.5122 - val_loss: 0.7114
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5642 - loss: 0.6829 - val_accuracy: 0.6098 - val_loss: 0.6656
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5642 - loss: 0.6829 - val_accuracy: 0.6098 - val_loss: 0.6656
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6497 - loss: 0.6370 - val_accuracy: 0.6423 - val_loss: 0.6307
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6497 - loss: 0.6370 - val

In [15]:
import keras_tuner as kt

In [16]:
def build_model(hp):
    """
    Build a neural network model with hyperparameters to tune.
    
    Hyperparameters to optimize:
    - Number of hidden nodes (16-128)
    - Optimizer type (Adam, SGD, RMSprop, Adadelta)  
    - Learning rate (0.0001-0.01)
    """
    model = Sequential()
    
    # Add input layer to avoid deprecation warnings
    model.add(Input(shape=(8,)))
    
    # Tune the number of hidden nodes
    hidden_nodes = hp.Int('hidden_nodes', min_value=16, max_value=128, step=16)
    model.add(Dense(hidden_nodes, activation='relu'))
    
    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Tune the optimizer type
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    
    # Tune the learning rate
    learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='LOG')
    
    # Configure the optimizer with the tuned learning rate
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adadelta':
        opt = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

print("Hyperparameter tuning function defined successfully!")

Hyperparameter tuning function defined successfully!


In [17]:
# Create hyperparameter tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  
    max_trials=5,
    directory='tuner_results',
    project_name='diabetes_hyperparameter_tuning'
)

print("Tuner created successfully!")

Reloading Tuner from tuner_results\diabetes_hyperparameter_tuning\tuner0.json
Tuner created successfully!


In [18]:
# Start the hyperparameter search
# This will try different combinations of hyperparameters to find the best ones
print("Starting hyperparameter search...")
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
print("Search completed!")

Starting hyperparameter search...
Search completed!


In [19]:
# Get the best hyperparameters found during the search
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print("Best hyperparameters found:")
print(f"Hidden nodes: {best_hyperparameters.get('hidden_nodes')}")
print(f"Optimizer: {best_hyperparameters.get('optimizer')}")
print(f"Learning rate: {best_hyperparameters.get('learning_rate'):.6f}")

Best hyperparameters found:
Hidden nodes: 64
Optimizer: adam
Learning rate: 0.000352


In [20]:
# Get the best model with the optimal hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
print("Best model retrieved successfully!")


Best model retrieved successfully!
Best model retrieved successfully!


c:\Users\Ayesh\OneDrive\Documents\uni\ML Summer\hyperparameter tuning\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
# Display the architecture of the best model
print("Best model architecture:")
best_model.summary()

Best model architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the best model with more epochs for better performance
print("Training the best model...")
history = best_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
print("Training completed!")

Training the best model...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7454 - loss: 0.5528 - val_accuracy: 0.7073 - val_loss: 0.5496
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7454 - loss: 0.5528 - val_accuracy: 0.7073 - val_loss: 0.5496
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7454 - loss: 0.5408 - val_accuracy: 0.7154 - val_loss: 0.5414
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7454 - loss: 0.5408 - val_accuracy: 0.7154 - val_loss: 0.5414
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7495 - loss: 0.5314 - val_accuracy: 0.7073 - val_loss: 0.5353
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7495 - loss: 0.5314 - val_accuracy: 0.7073 - val_loss: 0.5353
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7515 - loss: 0.5234 - val_accuracy: 0.7154 - val_loss: 0.5294
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7515 - loss: 0.5234 - val

In [23]:
# Evaluate the best model on test data
print("Evaluating the model on test data...")
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

Evaluating the model on test data...
Test Loss: 0.5240
Test Accuracy: 0.7468 (74.68%)
Test Loss: 0.5240
Test Accuracy: 0.7468 (74.68%)


## 📊 Results and Analysis

Now let's analyze our results and make predictions with the optimized model!

In [24]:
# Make predictions on test data and compare with actual labels
print("Making predictions on test data...")
predictions = best_model.predict(X_test, verbose=0)

# Convert probabilities to binary predictions (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

print("\n=== Prediction Results ===")
print(f"First 10 probability predictions: {predictions[:10].flatten()}")
print(f"First 10 binary predictions:     {binary_predictions[:10].flatten()}")
print(f"First 10 actual labels:          {y_test[:10]}")

# Calculate accuracy manually as verification
correct_predictions = (binary_predictions.flatten() == y_test).sum()
total_predictions = len(y_test)
manual_accuracy = correct_predictions / total_predictions
print(f"\nManual accuracy calculation: {correct_predictions}/{total_predictions} = {manual_accuracy:.4f} ({manual_accuracy*100:.2f}%)")

Making predictions on test data...

=== Prediction Results ===
First 10 probability predictions: [0.47535744 0.16253927 0.14054975 0.3385958  0.43902335 0.6000011
 0.05009026 0.6016964  0.5173687  0.562292  ]
First 10 binary predictions:     [0 0 0 0 0 1 0 1 1 1]
First 10 actual labels:          [0 0 0 0 0 0 0 0 0 0]

Manual accuracy calculation: 115/154 = 0.7468 (74.68%)

=== Prediction Results ===
First 10 probability predictions: [0.47535744 0.16253927 0.14054975 0.3385958  0.43902335 0.6000011
 0.05009026 0.6016964  0.5173687  0.562292  ]
First 10 binary predictions:     [0 0 0 0 0 1 0 1 1 1]
First 10 actual labels:          [0 0 0 0 0 0 0 0 0 0]

Manual accuracy calculation: 115/154 = 0.7468 (74.68%)


## 🎯 Summary and Conclusions

**Key Takeaways:**
1. **Hyperparameter tuning improved our model performance** by automatically finding optimal settings
2. **The process tested different combinations** of hidden layer sizes, optimizers, and learning rates
3. **Automated tuning saves time** compared to manual trial-and-error approaches
4. **The optimized model** achieved better results than our baseline model

**What we learned:**
- How to set up automated hyperparameter tuning with Keras Tuner
- The importance of feature scaling for neural networks
- How to compare model performance before and after optimization
- Best practices for neural network architecture and training

This approach can be applied to any neural network project to improve model performance! 🚀